In [ ]:
import mediapipe as mp      # I import this library because hand detection code code is already exist in this library we dont need to implement the further code
import cv2
import time                 # I implement this library because we have to maintain the frame per second record.if fps value increases it means our hand perfectly detected and fps value decreases means our hand not accurately detected
import os
import pyautogui as p       # I import this library because through gesture we want to control our keyboard


cap = cv2.VideoCapture(0)
cap.set(3, 780)                 # width , value
cap.set(4, 680)                  # height ,value

folderPath = "Fingers"
myList = os.listdir(folderPath)      # mylist variable store the finger file names
overLayList = []                     # OverLayList takes the path of the finger files 
for imPath in myList:
    image = cv2.imread(f"{folderPath}/{imPath}")
    overLayList.append(image)

mpHands = mp.solutions.hands               # to access those hands we use this method mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1)     # max 1 hand will be detected
mpDraw = mp.solutions.drawing_utils        # solutions darwing utils method will darw our hand landmark
cTime = 0                                   # Initially we declare oure current and previous time as 0
pTime = 0

tipId = [4, 8, 12, 16, 20]                # this is the Landmark number of our hand peaks
while True:                                
    success, img = cap.read()              
    img = cv2.flip(img, 1)                # the image will flip means rotate (1 means y axis,0 means x axis)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)         # our image will detect in RGB   
    result = hands.process(imgRGB)
    List1 = []                   # the landmark value is store in this empty list 
    if result.multi_hand_landmarks:      
        for handlms in result.multi_hand_landmarks:         # handlms is list      
            for id, lm in enumerate(handlms.landmark):         # id number lm ma landmarks k value hogi enumerate function will store the value of landmark
                h, w, c = img.shape                 # return the image height width and channel (color value k show karega)
                cx, cy = int(lm.x*w), int(lm.y*h)       # landmark value will be multiply by width of image and height of image to get the corrdinate on the basis of screen
                List1.append([id, cx, cy])              # cx cy to acees the landmark of the screen 
                cv2.circle(img, (cx, cy), 8, (255, 0, 0), cv2.FILLED)       # every landmark will make circle on it, 8 is radius, circle value will be filled
            # print(List1)
            mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)  # to detect the landmark dots in screen (handlms contains the landmark values )

    if len(List1) != 0:       # always contain any value of landmark 
        fingers = []
        # Thumb(list )
        if List1[tipId[0]][1] < List1[tipId[0] - 1][1]:     # 2 dimension id in list1 (0 will replace with 4 value also, 1 is used to access the landmark peak thumb ,after less than code is used to access the landmark values after peak values and peak value is low and next to peak values is always high) 
            fingers.append(1)
        else:
            fingers.append(0)

        # 4 Fingers
        for id in range(1, 5):  
            if List1[tipId[id]][2] < List1[tipId[id]-2][2]:    # 2 number will access the 1 finger peak value that is 8 and after less than code it will subtract the landmark value like 8-2=6 6 landmark value will be access 
                fingers.append(1)
            else:
                fingers.append(0)

        fingers = fingers.count(1)        # finger.count is a list, it will count our fingers and store in the fingers variable but here we set count value 1 
        print(fingers)
        h, w, c = overLayList[fingers-1].shape           # fingers is simply the count to access the image of hand 
        img[0:h, 0:w] = overLayList[fingers-1]           

        cv2.rectangle(img, (10, 320), (160, 500), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, f'{int(fingers)}', (30, 460), cv2.FONT_HERSHEY_PLAIN, 10, (255, 0, 0), 10)

         # Conditions controlling video
#         if fingers == 0:
#             cv2.putText(img, " ", (250, 50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (0, 0, 255), 2)
#         elif fingers == 1:
#             p.press("up")
#             cv2.putText(img, "Volume UP", (250, 50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (0, 0, 255), 2)
#         elif fingers == 2:
#             p.press("down")
#             cv2.putText(img, "Volume Down", (220, 50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (0, 0, 255), 2)
#         elif fingers == 3:
#             p.press("left")
#             cv2.putText(img, "Backward", (250, 50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (0, 0, 255), 2)
#         elif fingers == 4:
#             p.press("right")
#             cv2.putText(img, "Forward", (250, 50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (0, 0, 255), 2)
#         elif fingers == 5:
#             p.press("space")
#             cv2.putText(img, "Play/Pause", (250, 50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (0, 0, 255), 2)
#         else:
#             pass

    cTime = time.time()    # current time and previous time  will tell us the system time actually
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img, f' fps: {int(fps)}', (600, 60), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)
    cv2.imshow('Frames', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()